In [ ]:
# --- [ 1. IMPORTS ] ---
print("Importing libraries...")
import pandas as pd
import numpy as np
import ast  # For safely evaluating the 'apps' string
import json # Fallback for parsing
import re   # For cleaning app names (Regular Expressions)
import csv  # For robust CSV loading
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import display, HTML

# --- [ 2. SETUP & HELPER FUNCTIONS ] ---
tqdm.pandas(desc="Processing...")

def parse_apps_string(apps_str):
    if pd.isna(apps_str): return []
    try:
        return ast.literal_eval(apps_str)
    except (ValueError, SyntaxError):
        try:
            return json.loads(apps_str.replace("'", "\""))
        except:
            return []

def infer_manufacturer(process_name):
    process_name = str(process_name).lower()
    if 'com.samsung' in process_name: return 'Samsung'
    if 'com.apple' in process_name or 'ios' in process_name: return 'Apple'
    if 'com.google' in process_name: return 'Google/Stock'
    if 'com.htc' in process_name: return 'HTC'
    if 'com.motorola' in process_name: return 'Motorola'
    if 'com.sony' in process_name: return 'Sony'
    if 'com.oneplus' in process_name: return 'OnePlus'
    if 'com.huawei' in process_name: return 'Huawei'
    return np.nan

def get_simple_app_name(process_name):
    try:
        process_name = str(process_name)
        parts = process_name.split('.')
        if len(parts) > 1:
            if parts[0] == 'com' and len(parts) > 2:
                name = parts[1]
                if name in ['android', 'google', 'samsung', 'htc', 'motorola']:
                     if len(parts) > 3: return parts[3]
                     elif len(parts) > 2: return parts[2]
                return name
            else:
                return parts[0]
        return process_name
    except:
        return 'unknown'

# --- [ 3. DATA LOADING, PREPARATION, & FILTERING ] ---
print("\n--- [ STAGE 1: DATA LOADING & PREPARATION ] ---")
print("Using 'bulletproof' CSV loading method...")

try:
    expected_columns = [
        'uuid', 'timestamp', 'batteryLevel', 'batteryStatus',
        'timeZone', 'mobileCountryCode', 'apps'
    ]
    data_list = []

    with open('mega_dataset.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f, quotechar='"')
        header = next(reader)
        print(f"File header found: {header}")

        for row in tqdm(reader, desc="Reading file line by line"):
            if len(row) == len(expected_columns):
                row_dict = dict(zip(expected_columns, row))
                data_list.append(row_dict)

    print(f"\nSuccessfully read {len(data_list)} rows from CSV.")

    df_raw = pd.DataFrame(data_list)

    if df_raw.empty or 'apps' not in df_raw.columns:
        raise ValueError("Manual CSV parsing failed.")
    if df_raw['apps'].isna().all() or (df_raw['apps'] == '').all():
        raise ValueError("'apps' column is empty. Check file corruption or formatting.")

    print(f"Raw data loaded into DataFrame. Shape: {df_raw.shape}")
    display(df_raw.head())

    print("\nParsing the complex 'apps' string...")
    df_raw['apps_list'] = df_raw['apps'].progress_apply(parse_apps_string)

    print("Exploding data... (each app gets its own row)")
    df_exploded = df_raw.explode('apps_list')
    df_exploded = df_exploded.dropna(subset=['apps_list'])
    df_exploded = df_exploded[df_exploded['apps_list'].apply(lambda x: isinstance(x, dict))]
    print(f"Data 'exploded'. New shape: {df_exploded.shape}")

    if df_exploded.empty:
        raise ValueError("No app data found after parsing 'apps' column.")

    print("Normalizing app data (splitting dictionaries to columns)...")
    app_data = pd.json_normalize(df_exploded['apps_list'])
    df_exploded = df_exploded.reset_index(drop=True)
    app_data = app_data.reset_index(drop=True)
    df_clean = pd.concat([df_exploded, app_data], axis=1)

    print("Engineering new features...")
    df_clean['timestamp'] = pd.to_numeric(df_clean['timestamp'], errors='coerce')
    df_clean['batteryLevel'] = pd.to_numeric(df_clean['batteryLevel'], errors='coerce')
    df_clean = df_clean.dropna(subset=['timestamp'])
    df_clean['datetime'] = pd.to_datetime(df_clean['timestamp'], unit='s')
    df_clean['hour_of_day'] = df_clean['datetime'].dt.hour
    df_clean['day_of_week'] = df_clean['datetime'].dt.day_name()
    df_clean['deviceManufacturer'] = df_clean['processName'].progress_apply(infer_manufacturer)
    df_clean['simpleAppName'] = df_clean['processName'].progress_apply(get_simple_app_name)

    df_clean = df_clean.drop(columns=['apps', 'apps_list'])
    df_clean['processName'] = df_clean['processName'].fillna('unknown')
    df_clean['priority'] = df_clean['priority'].fillna('unknown')

    print(f"\n--- [ STAGE 2: PRE-ANALYSIS FILTERING ] ---")
    print(f"Data shape before filtering: {df_clean.shape}")

    df_clean = df_clean.dropna(subset=['deviceManufacturer'])
    print(f"Data shape after filtering manufacturers: {df_clean.shape}")

    print("Identifying real user apps (apps that run in the foreground)...")
    foreground_app_names = df_clean[
        df_clean['priority'] == 'Foreground app'
    ]['simpleAppName'].unique()

    if len(foreground_app_names) == 0:
        print("--- [ WARNING ] --- No 'Foreground app' processes found. Analysis may fail.")
        df_clean = pd.DataFrame(columns=df_clean.columns)
    else:
        df_user_apps = df_clean[df_clean['simpleAppName'].isin(foreground_app_names)]
        top_20_user_app_names = df_user_apps['simpleAppName'].value_counts().head(20).index

        print(f"\nTop 20 user-installed apps identified for analysis:")
        print(top_20_user_app_names.to_list())

        df_clean = df_clean[df_clean['simpleAppName'].isin(top_20_user_app_names)]
        print(f"Data shape after filtering for Top 20 apps: {df_clean.shape}")

    print("\n--- DATA PREPARATION & FILTERING COMPLETE ---")
    print("\nClean data preview:")
    display(df_clean.head())

except FileNotFoundError:
    print("--- [ FATAL ERROR ] ---")
    print("ERROR: 'mega_dataset.csv' not found.")
    print("Please upload the dataset to your Google Colab environment and try again.")
except Exception as e:
    print(f"--- [ FATAL ERROR ] ---")
    print(f"An unexpected error occurred during data preparation: {e}")
    print("Please check the dataset file and the code.")
    df_clean = pd.DataFrame()

# --- [ ANALYSIS STAGES - NEW MATPLOTLIB/SEABORN VERSION ] ---

if 'df_clean' in locals() and not df_clean.empty:

    # Set default plot theme and style
    sns.set_theme(style="whitegrid", palette="muted")

    # ---
    print("\n\n--- [ STAGE 3: ANALYSIS for 'Performance & Battery Diagnostics' ] ---")
    # ---
    print("Analyzing battery status and levels (for Top 20 apps)...")

    # Chart 1: Battery Status Distribution
    plt.figure(figsize=(10, 6))
    status_counts = df_clean['batteryStatus'].value_counts()
    plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', startangle=90,
            wedgeprops=dict(width=0.4)) # Donut chart
    plt.title('[Top 20 Apps] Overall Battery Status (Charging vs. Discharging)')
    plt.gca().add_artist(plt.Circle((0,0),0.2,fc='white')) # Add center circle
    plt.show()

    # Chart 2: Battery Level Distribution
    plt.figure(figsize=(12, 6))
    sns.histplot(df_clean, x='batteryLevel', bins=100, kde=True)
    plt.title('[Top 20 Apps] Distribution of Battery Levels')
    plt.xlabel('Battery Level (%)')
    plt.ylabel('Count')
    plt.show()

    # Chart 3: Top 20 Most Frequent Apps (Overall)
    print("Calculating frequencies for Top 20 apps...")
    plt.figure(figsize=(12, 8))
    top_20_apps = df_clean['simpleAppName'].value_counts()
    sns.barplot(x=top_20_apps.values, y=top_20_apps.index, orient='h')
    plt.title('[Top 20 Apps] Frequency of Running Apps')
    plt.xlabel('Count')
    plt.ylabel('App Name')
    plt.show()

    print("\n\n--- [ STAGE 4: ANALYSIS for 'Smart Battery Regulation' ] ---")
    print("Analyzing apps running during 'discharging' state (for Top 20 apps)...")

    df_discharging = df_clean[df_clean['batteryStatus'] != 'charging'].copy()

    if df_discharging.empty:
        print("No 'discharging' data found. Skipping this section.")
    else:
        # Chart 4: Top 20 Apps Running While Battery is Discharging
        plt.figure(figsize=(12, 8))
        top_20_discharging = df_discharging['simpleAppName'].value_counts()
        sns.barplot(x=top_20_discharging.values, y=top_20_discharging.index, orient='h', color='orange')
        plt.title('[Top 20 Apps] Frequency While Battery is Discharging')
        plt.xlabel('Count')
        plt.ylabel('App Name')
        plt.show()

        # Chart 5: Analysis of 'priority' levels
        plt.figure(figsize=(10, 7))
        priority_counts = df_clean['priority'].value_counts()
        plt.pie(priority_counts, labels=priority_counts.index, autopct='%1.1f%%',
                startangle=90, wedgeprops=dict(width=0.4))
        plt.title('[Top 20 Apps] Distribution of App Process Priorities')
        plt.gca().add_artist(plt.Circle((0,0),0.2,fc='white'))
        plt.show()

        # Chart 6: THE "HIT LIST" - Top 20 Background Apps Draining Battery
        background_priorities = ['Service', 'Background process']
        df_bg_drain = df_discharging[df_discharging['priority'].isin(background_priorities)].copy()

        if df_bg_drain.empty:
            print("No background draining apps found (for Top 20 apps).")
        else:
            plt.figure(figsize=(12, 8))
            top_20_bg_drain = df_bg_drain['simpleAppName'].value_counts()
            sns.barplot(x=top_20_bg_drain.values, y=top_20_bg_drain.index, orient='h', color='red')
            plt.title('[Top 20 Apps] HIT LIST: Background Draining Frequency')
            plt.xlabel('Count')
            plt.ylabel('App Name')
            plt.show()

    print("\n\n--- [ STAGE 5: ANALYSIS for 'Intelligent Resource Allocation' ] ---")
    print("Analyzing app usage patterns by time of day (for Top 20 apps)...")

    # Chart 7: App Usage by Hour of Day (Heatmap)
    plt.figure(figsize=(16, 10))
    app_hour_pivot = pd.crosstab(df_clean['simpleAppName'], df_clean['hour_of_day'])
    sns.heatmap(app_hour_pivot, cmap="viridis", linewidths=.5)
    plt.title('[Top 20 Apps] Heatmap of App Usage by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('App Name')
    plt.show()

    # Chart 8: Foreground vs. Background by Hour
    def simplify_priority(p):
        if p == 'Foreground app': return 'Foreground'
        if p in ['Service', 'Background process']: return 'Background'
        return 'Other'

    df_clean['simplePriority'] = df_clean['priority'].apply(simplify_priority)
    priority_hour_simple = df_clean.groupby(['hour_of_day', 'simplePriority']).size().unstack().fillna(0)

    plt.figure(figsize=(14, 7))
    # Use pandas' built-in plotting which uses matplotlib
    priority_hour_simple.plot(kind='bar', stacked=True, figsize=(14, 7))
    plt.title('[Top 20 Apps] App Usage Type by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Count')
    plt.legend(title='Priority Type')
    plt.show()

    # ---
    print("\n\n--- [ STAGE 6: ANALYSIS for 'Adaptive Connectivity Management' ] ---")
    # ---
    print("Analyzing user distribution by TimeZone and Country Code (for Top 20 apps)...")

    # Chart 9: Top 20 TimeZones
    plt.figure(figsize=(12, 8))
    top_20_tz = df_clean['timeZone'].value_counts().head(20)
    sns.barplot(x=top_20_tz.values, y=top_20_tz.index, orient='h')
    plt.title('[Top 20 Apps] Top 20 User TimeZones')
    plt.xlabel('Count')
    plt.ylabel('TimeZone')
    plt.show()

    # Chart 10: Top 20 Mobile Country Codes (MCC)
    plt.figure(figsize=(12, 8))
    top_20_mcc = df_clean['mobileCountryCode'].value_counts().head(20)
    sns.barplot(x=top_20_mcc.values, y=top_20_mcc.index, orient='h')
    plt.title('[Top 20 Apps] Top 20 Mobile Country Codes (MCC)')
    plt.xlabel('Count')
    plt.ylabel('MCC')
    plt.show()

    print("Insight for Connectivity: High 'unknown' MCC counts could mean Wi-Fi-only use.")

    print("\n\n--- [ STAGE 7: ANALYSIS by Phone Model (Inferred Manufacturer) ] ---")
    print("Analyzing differences between inferred device manufacturers (for Top 20 apps)...")

    # Chart 11: Distribution of Inferred Manufacturers
    def clean_manufacturer(m):
        if m in ['Samsung', 'Google/Stock', 'Apple']: return m
        if pd.isna(m): return 'Unknown'
        return 'Other'

    df_clean['deviceManuf_simple'] = df_clean['deviceManufacturer'].apply(clean_manufacturer)
    manuf_counts = df_clean['deviceManuf_simple'].value_counts()

    plt.figure(figsize=(10, 6))
    plt.pie(manuf_counts, labels=manuf_counts.index, autopct='%1.1f%%',
            startangle=90, wedgeprops=dict(width=0.4))
    plt.title('[Top 20 Apps] Inferred Device Manufacturer Distribution')
    plt.gca().add_artist(plt.Circle((0,0),0.2,fc='white'))
    plt.show()

    # Chart 12: Top Background Draining Apps by Manufacturer
    if 'df_bg_drain' in locals() and not df_bg_drain.empty:
        df_bg_drain['deviceManuf_simple'] = df_bg_drain['deviceManufacturer'].apply(clean_manufacturer)
        manuf_list = ['Samsung', 'Google/Stock', 'Other']
        df_bg_drain_top_manuf = df_bg_drain[df_bg_drain['deviceManuf_simple'].isin(manuf_list)]

        if not df_bg_drain_top_manuf.empty:
            top_apps_by_manuf = df_bg_drain_top_manuf.groupby('deviceManuf_simple')['simpleAppName'].value_counts().groupby(level=0).head(5).to_frame(name='count').reset_index()

            plt.figure(figsize=(14, 7))
            sns.barplot(x='deviceManuf_simple', y='count', hue='simpleAppName', data=top_apps_by_manuf)
            plt.title('[Top 20 Apps] Top 5 Background Draining Apps by Manufacturer')
            plt.xlabel('Manufacturer')
            plt.ylabel('Frequency')
            plt.legend(title='App Name', bbox_to_anchor=(1.05, 1), loc=2)
            plt.tight_layout()
            plt.show()

            print("--- [ ANALYSIS COMPLETE ] ---")
        else:
            print("No background draining apps found for the main manufacturers (Samsung, Google, Other).")
    else:
        print("Skipping Chart 12: No background draining apps found in the filtered dataset.")

else:
    print("\nAnalysis was skipped due to an error in data loading/preparation or the dataset being empty after filtering.")